In [ ]:
###########################################
PROJECT_NAME = 'CFD_Pipe'
###########################################

In [ ]:
!pip install openai==0.28
!apt-get install -y libglu1-mesa
!pip install gmsh
!pip install pyvista

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libglu1-mesa is already the newest version (9.0.2-1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
import openai
import os

In [ ]:
openai.api_key =("Input key here")

In [ ]:
# Global variables to keep track of the conversation
conversation_history = []
last_response = None

# Read the system message once and append to history
with open("system_geo.txt", 'r') as file:
    system_message = file.read().strip()
conversation_history.append({"role": "system", "content": system_message})

In [ ]:
# Function to interact with OpenAI's API
def chat_with_bot(user_message):
    global last_response
    # Add the user input to the conversation history
    conversation_history.append({"role": "user", "content": user_message})

    # Call the updated OpenAI API (correct method)
    response = openai.ChatCompletion.create(
        model="gpt-4",  # You can choose the model here (e.g., gpt-4, gpt-3.5-turbo)
        messages=conversation_history,
        temperature=0
    )

    # Extract the model's response
    model_response = response['choices'][0]['message']['content']

    # Update the last response
    last_response = model_response

    # Append the model's response to the conversation history
    conversation_history.append({"role": "assistant", "content": model_response})

    # Print the model's response
    print(model_response)

In [ ]:
# Function to save the response as a .geo file
def extract_and_save_geo_file(response_text):
    file_path = f"CFD_Pipe.geo"
    try:
        with open(file_path, 'w') as geo_file:
            geo_file.write(response_text)
        print(f"{file_path} file saved")
        print("Visualize .geo files with GMSH")
    except Exception as e:
        print(f"Error saving the file: {e}")

In [ ]:
#FUNCTIONS FROM CREATING THE MESH ONCE THE .GEO FILE
import gmsh

def generate_mesh(Mesh_Min_Size, Mesh_Max_Size):
    print("Creating MESH...\n ")
    try:
        current_directory = os.getcwd()
        input_geo_file = os.path.join(current_directory, f'CFD_Pipe.geo')
        output_msh_file = os.path.join(current_directory, f'CFD_Pipe.msh')
        # Initialize gmsh
        gmsh.initialize()
        # Open the .geo file
        gmsh.open(input_geo_file)
        gmsh.option.setNumber("Mesh.CharacteristicLengthMin", Mesh_Min_Size)
        gmsh.option.setNumber("Mesh.CharacteristicLengthMax", Mesh_Max_Size)
        # Generate the mesh using default settings
        gmsh.model.mesh.generate(3)
        # Save the mesh to a .msh file
        gmsh.write(output_msh_file)
        # Finalize gmsh
        gmsh.finalize()
        print(f"CFD_Pipe.msh saved")
        print("You can visualize .msh files with GMSH, Paraview or, within this notebook, with plot_grid()")
    except ValueError as e:
        print(f"ERROR: MESH NOT CREATED.")

#(optional)
import pyvista as pv

def plot_grid(output_msh_file):
    # Read mesh file
    mesh = pv.read(output_msh_file)
    # Set the Jupyter backend to 'static'
    pv.set_jupyter_backend('static')
    # Plot the mesh
    plotter = pv.Plotter(notebook=True)
    plotter.add_mesh(mesh, show_edges=True)
    plotter.show()

In [ ]:
############ INSTRUCTIONS #######################
# Chat with model with chat_with_bot("..."),
# when you are satisfied save the .geo file with
# extract_and_save_geo_file(last_response) and
# visualize it with gmsh software.
# Create mesh with generate_mesh() and
# visualize it with gmsh, Paraview, or
# plot_grid(msh_file) for quick visualization
###############################################

In [ ]:
chat_with_bot("""Create a pipe. The axis of the bar is parallel to the z axis
""")

Sure, I will need some more information to create the .geo file for the pipe. Could you please provide the length and radius of the pipe?


In [ ]:
chat_with_bot("The length of pipe is 6m and the radius is 30.165 mm")

Great! Here is the .geo file for a straight pipe with a length of 6 meters and a radius of 30.165 mm.

//BEGIN_GEO
lc = 1; // characteristic length
Length = 6; // length of the pipe
Radius = 0.030165; // radius of the pipe

// Define the points
Point(1) = {0, 0, 0, lc}; // inlet
Point(2) = {0, 0, Length, lc}; // outlet

// Define the lines
Line(1) = {1, 2}; // pipe

// Define the surfaces
Line Loop(1) = {1};
Plane Surface(1) = {1}; // pipe wall

// Define the volume
Surface Loop(1) = {1};
Volume(1) = {1}; // fluid volume inside the pipe

// Define physical groups
Physical Surface("Inlet") = {1};
Physical Surface("Outlet") = {2};
Physical Surface("Wall") = {1};
Physical Volume("FluidDomain") = {1};
//END_GEO


In [ ]:
chat_with_bot("You didnt ask me if the simulation is 2D or 3D")

I apologize for the oversight. Could you please specify whether the simulation is 2D or 3D?


In [ ]:
chat_with_bot("The simulation is 3D")

Thank you for the clarification. Here is the revised .geo file for a 3D pipe with a length of 6 meters and a radius of 30.165 mm.

//BEGIN_GEO
lc = 1; // characteristic length
Length = 6; // length of the pipe
Radius = 0.030165; // radius of the pipe

// Define the points
Point(1) = {0, 0, 0, lc}; // inlet
Point(2) = {0, 0, Length, lc}; // outlet

// Define the lines
Line(1) = {1, 2}; // pipe

// Define the surfaces
Circle(1) = {1, 2, Radius}; // pipe wall
Line Loop(1) = {1};
Ruled Surface(1) = {1}; // pipe wall

// Define the volume
Surface Loop(1) = {1};
Volume(1) = {1}; // fluid volume inside the pipe

// Define physical groups
Physical Surface("Inlet") = {1};
Physical Surface("Outlet") = {2};
Physical Surface("Wall") = {1};
Physical Volume("FluidDomain") = {1};
//END_GEO


In [ ]:
chat_with_bot("The simulation shows alot of errors :Error   : Line with less than 2 control points")

I apologize for the confusion. The error is due to the incorrect definition of the pipe's geometry. In a 3D simulation, we need to define the pipe as a cylindrical surface rather than a line. Here is the corrected .geo file:

//BEGIN_GEO
lc = 1; // characteristic length
Length = 6; // length of the pipe
Radius = 0.030165; // radius of the pipe

// Define the points
Point(1) = {0, 0, 0, lc}; // inlet
Point(2) = {0, 0, Length, lc}; // outlet
Point(3) = {Radius, 0, 0, lc}; // point on the pipe wall at inlet
Point(4) = {Radius, 0, Length, lc}; // point on the pipe wall at outlet

// Define the lines
Circle(1) = {1, 3, 2}; // pipe wall at inlet
Circle(2) = {3, 4, 2}; // pipe wall at outlet
Line(3) = {3, 4}; // pipe wall

// Define the surfaces
Line Loop(1) = {1, -2, 3};
Ruled Surface(1) = {1}; // pipe wall

// Define the volume
Surface Loop(1) = {1};
Volume(1) = {1}; // fluid volume inside the pipe

// Define physical groups
Physical Surface("Inlet") = {1};
Physical Surface("Outlet") = {2};

In [ ]:
chat_with_bot("Error   : Circle or ellipse with less than 2 control points")

I apologize for the confusion. The error is due to the incorrect definition of the pipe's geometry. In a 3D simulation, we need to define the pipe as a cylindrical surface rather than a line. Here is the corrected .geo file:

//BEGIN_GEO
lc = 1; // characteristic length
Length = 6; // length of the pipe
Radius = 0.030165; // radius of the pipe

// Define the points
Point(1) = {0, 0, 0, lc}; // inlet center
Point(2) = {0, 0, Length, lc}; // outlet center
Point(3) = {Radius, 0, 0, lc}; // point on the pipe wall at inlet
Point(4) = {Radius, 0, Length, lc}; // point on the pipe wall at outlet

// Define the lines
Circle(1) = {1, 3, 4}; // pipe wall
Line(2) = {3, 4}; // pipe axis

// Define the surfaces
Line Loop(1) = {1, 2};
Ruled Surface(1) = {1}; // pipe wall

// Define the volume
Surface Loop(1) = {1};
Volume(1) = {1}; // fluid volume inside the pipe

// Define physical groups
Physical Surface("Inlet") = {1};
Physical Surface("Outlet") = {2};
Physical Surface("Wall") = {1};
Physical Vol

In [ ]:
chat_with_bot("The simulation shows alot of errors :Error   : Line with less than 2 control points")

I apologize for the confusion. The error is due to the incorrect definition of the pipe's geometry. In a 3D simulation, we need to define the pipe as a cylindrical surface rather than a line. Here is the corrected .geo file:

//BEGIN_GEO
lc = 1; // characteristic length
Length = 6; // length of the pipe
Radius = 0.030165; // radius of the pipe

// Define the points
Point(1) = {0, 0, 0, lc}; // inlet center
Point(2) = {0, 0, Length, lc}; // outlet center
Point(3) = {Radius, 0, 0, lc}; // point on the pipe wall at inlet
Point(4) = {Radius, 0, Length, lc}; // point on the pipe wall at outlet

// Define the lines
Circle(1) = {1, 3, 4}; // pipe wall
Line(2) = {3, 4}; // pipe axis

// Define the surfaces
Line Loop(1) = {1, 2};
Ruled Surface(1) = {1}; // pipe wall

// Define the volume
Surface Loop(1) = {1};
Volume(1) = {1}; // fluid volume inside the pipe

// Define physical groups
Physical Surface("Inlet") = {1};
Physical Surface("Outlet") = {2};
Physical Surface("Wall") = {1};
Physical Vol

In [ ]:
extract_and_save_geo_file(last_response)

CFD_Pipe.geo file saved
Visualize .geo files with GMSH


In [ ]:
generate_mesh(Mesh_Min_Size = 0.03, Mesh_Max_Size = 0.2)

Creating MESH...
 
CFD_Pipe.msh saved
You can visualize .msh files with GMSH, Paraview or, within this notebook, with plot_grid()
